# RNN Character Language Model

In [1]:
import sys,random,math
from collections import Collection
import numpy as np
import sys
from Chapter_13.tensor import Tensor
from Chapter_13.layers.layers import Embedding,RNNCell
from Chapter_13.loss.loss import CrossEntropyLoss
from Chapter_13.optmizers.gradient_descent import SGD
np.random.seed(0)

<ipython-input-1-b8d5854570fb>:2: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Collection


In [2]:
f = open("shakespear.txt")
raw = f.read()
f.close()

vocab = list(set(raw))
word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i
indices = np.array(list(map(lambda x:word2index[x], raw)))

In [3]:
embed = Embedding(vocab_size=len(vocab),dim=512)
model = RNNCell(n_input=512,n_hidden=512,n_output=len(vocab))
criterion = CrossEntropyLoss()
optim = SGD(model.get_parameters()+embed.get_parameters(),alpha=0.05)

In [4]:
batch_size = 32
bptt = 16
n_batches = int((indices.shape[0] / (batch_size)))

trimmed_indices = indices[:n_batches*batch_size]
batched_indices = trimmed_indices.reshape(batch_size, n_batches).transpose()

input_batched_indices = batched_indices[0:-1]
target_batched_indices = batched_indices[1:]

n_bptt = int(((n_batches-1) / bptt))
input_batches = input_batched_indices[:n_bptt*bptt].reshape(n_bptt,bptt,batch_size)
target_batches = target_batched_indices[:n_bptt*bptt].reshape(n_bptt, bptt, batch_size)

In [5]:
print(raw[0:5])
print(indices[0:5])

That,
[37 50 36 21 18]


In [6]:
print(batched_indices[0:5])

[[37 31 19 40 46 50 37 12 12 36 36 39 46 12 18  5 12 43 50 19 39 36 12 12
  12 60 12 60 60 12 12  9]
 [50  5  5 40  1 39 36 51 43  9 45 39 45 21 12  5 55 46 46  5 45 25 55 29
  50  9  8  4  9  4 56 12]
 [36  5 14 39 39 36 45 10 46 12 45 19 29 50 28 49 39 51 45  5 29 39 38 39
  39 16 46 18 12 46 46 55]
 [21 57 37 38 12 38 55 12 51  9 12 12 31 39 39 47 39 18 43 48 10 12 46 39
  38 45 51  5 32 38  4 46]
 [18 47 60 14 28 12 46 38 12 46 28 24  5 12 12 52  9 12 12 46 31 43 28  1
  18 36 45 47 51 39 39 43]]


In [7]:
print(input_batches[0:5])

[[[37 31 19 ... 12 12  9]
  [50  5  5 ...  4 56 12]
  [36  5 14 ... 46 46 55]
  ...
  [ 9  0 39 ... 21 46 12]
  [21 19  8 ... 50 36 60]
  [39  5 21 ... 12 21  9]]

 [[ 4 48 12 ...  1 50 12]
  [ 1 46 10 ... 36 31 50]
  [21 28 60 ... 60  5 60]
  ...
  [14 46 10 ...  9  5 39]
  [29  9 21 ... 12 47 29]
  [12 39 12 ... 46 43 60]]

 [[21 12 40 ... 30 18  9]
  [50 21 60 ... 39 12 26]
  [46 50  9 ... 38 36 16]
  ...
  [12 12 38 ... 29 12 12]
  [40 11 39 ... 12 51 60]
  [36 12 12 ...  9  1  9]]

 [[60 28 60 ... 46 46 12]
  [21 46 10 ... 21  9 39]
  [50 51 12 ... 12 12 60]
  ...
  [12 12 38 ... 50 43 39]
  [ 8 36 12 ... 46 12 12]
  [46 12 36 ... 38 28 55]]

 [[ 9  4 45 ... 10 46 38]
  [21 60 60 ... 39 38 46]
  [38  9 25 ... 12 45 51]
  ...
  [21 36 32 ... 12 28 46]
  [50 43 37 ... 40 39 28]
  [39 12 23 ... 39 38 12]]]


In [8]:
print(target_batches[0:5])

[[[50  5  5 ...  4 56 12]
  [36  5 14 ... 46 46 55]
  [21 57 37 ... 38  4 46]
  ...
  [21 19  8 ... 50 36 60]
  [39  5 21 ... 12 21  9]
  [ 4 48 12 ...  1 50 12]]

 [[ 1 46 10 ... 36 31 50]
  [21 28 60 ... 60  5 60]
  [18 33  9 ...  9  5 10]
  ...
  [29  9 21 ... 12 47 29]
  [12 39 12 ... 46 43 60]
  [21 12 40 ... 30 18  9]]

 [[50 21 60 ... 39 12 26]
  [46 50  9 ... 38 36 16]
  [51 39 29 ... 16 43 28]
  ...
  [40 11 39 ... 12 51 60]
  [36 12 12 ...  9  1  9]
  [60 28 60 ... 46 46 12]]

 [[21 46 10 ... 21  9 39]
  [50 51 12 ... 12 12 60]
  [40 45 10 ... 10 46 26]
  ...
  [ 8 36 12 ... 46 12 12]
  [46 12 36 ... 38 28 55]
  [ 9  4 45 ... 10 46 38]]

 [[21 60 60 ... 39 38 46]
  [38  9 25 ... 12 45 51]
  [60 51 39 ... 11 29 26]
  ...
  [50 43 37 ... 40 39 28]
  [39 12 23 ... 39 38 12]
  [38 60 27 ... 36 12 21]]]


In [9]:
def generate_sample(n=30, init_char=' '):
    s = ""
    hidden = model.init_hidden(batch_size=1)
    input = Tensor(np.array([word2index[init_char]]))
    for i in range(n):
        rnn_input = embed.forward(input)
        output, hidden = model.forward(input=rnn_input, hidden=hidden)
        output.data *= 10
        temp_dist = output.softmax()
        temp_dist /= temp_dist.sum()

        m = (temp_dist > np.random.rand()).argmax()
#         m = output.data.argmax()
        c = vocab[m]
        input = Tensor(np.array([m]))
        s += c
    return s
print(generate_sample(n=2000, init_char='\n'))

YQGQYQYYYQYQYQGQQQQQYQYQQQQYYQQYQYQYQYQYQGGQYQYQYQYQQYQGYQYQYQYQQYYQQYQQQQYQQYQYYQYYQQQYQYQGQYQYQQYQQYQQYQYQYQQQYYQQQQQQYQYQYQYQQQQYQYQGQQYQYQYQQYQYQYQQGYQGGGYQQQQQQQYQQQQYYQQQQQQYYQQYYQYQYQGYQYYQQYQQYQQYYQQYQYQYQYQGQQYQGYQYQQQYQQYYQYQYQYQYQGQYQYQYQYQQQYQGYQQQYQQYYQQQQYQQYQQYQYQYQYQQYQYQYQYQGYQQQQQYQGYQYYYYYQQQYQQQQYQQQQYQYQYQYQYQYQGGYQGQQYYQQQQQQYQQQYQYQYYYQGGYYQQYQGGQGYYQYQGGQQGQYQQYQQQYQYQYQYQQYQYQQQQYQYQQYQYQYYQQYQQYQQYQYQQQQYQYQQQYQGGYQYQYQYQQQQYQQYQGGYYQQQYQYQYQYYQYYQYQQYQQYQYQQYQGGQQYQGGGGYQYQGYYQYQQQYQYQQQYQYQGQQYQGYQYQGYQYQQGYQQYQGYQQGYQYYQYQYQYQQQQYQYQQYQQQYQQYYQQQYYQYQYQYQGYQYQYQQYQYQQYQQYQGQYYQGQYQQYQYQQYQYQYQYQGGQYQQQYQYQYQQYQGYQQQQQYQYQYQQYQYQQGYYQQQQQQYQYQGGQYQGQYQYQYQQQYQYQYQYQQGQYQYQYQYQGQYQYQYQQYQGGYQYQQQYQGGQYQYQGYQQYQQYQYYQYQGQYYYQGYQYQYQYQQQQYQQQQGQYQYQYQQYQQYQYQYQQYQQGGQQYQQQYQYQYYQYQYQGQQYQYQYQQYQYQQQYQQYQQQQQYQGGYQGQQGQQYQYQYQQQYQYQYQYQGGYQGQYQYQYQYQGGYYQYQQYYYQGYQYQGYQYQQYQYQQYQQGYQGGGYQQQYQGQQYQQQYQYYQQQQYQYQYQQQYQQGQYYQYYQYQQYQQYQYQQGYQYYQYQGGQYYQYQYQYQYQGYQY

In [10]:
def train(iterations=400):
    for iter in range(iterations):
        total_loss = 0
        n_loss = 0

        hidden = model.init_hidden(batch_size=batch_size)
        for batch_i in range(len(input_batches)):

            hidden = Tensor(hidden.data, autograd=True)
            loss = None
            losses = list()
            for t in range(bptt):
                input = Tensor(input_batches[batch_i][t], autograd=True)
                rnn_input = embed.forward(input=input)
                output, hidden = model.forward(input=rnn_input, hidden=hidden)

                target = Tensor(target_batches[batch_i][t], autograd=True)
                batch_loss = criterion.forward(output, target)
                losses.append(batch_loss)
                if(t == 0):
                    loss = batch_loss
                else:
                    loss = loss + batch_loss
            for loss in losses:
                ""
            loss.backward()
            optim.step()
            total_loss += loss.data
            log = "\r Iter:" + str(iter)
            log += " - Batch "+str(batch_i+1)+"/"+str(len(input_batches))
            log += " - Loss:" + str(np.exp(total_loss / (batch_i+1)))
            if(batch_i == 0):
                log += " - " + generate_sample(n=70, init_char='\n').replace("\n"," ")
            if(batch_i % 10 == 0 or batch_i-1 == len(input_batches)):
                sys.stdout.write(log)
        optim.alpha *= 0.99
        print()

In [14]:
train(200)

 Iter:0 - Batch 191/195 - Loss:3.9091070364026326ThYerd the tYesY the the the Yarst to the the tYest to the the the to 
 Iter:1 - Batch 191/195 - Loss:3.8390553843301007This, a the to the to the tYest to the YarsY the the to the to the to 
 Iter:2 - Batch 191/195 - Loss:3.7643283883435803This, a the to the to the tYeY the Yarst to the to the tYeY the to the
 Iter:3 - Batch 191/195 - Loss:3.6691792363681364 ThY the the the the the the the the the the the tYest to the the the t
 Iter:4 - Batch 191/195 - Loss:3.6236716012921096This, a the the the the the the Yarse Yarse the to the tYest to the th
 Iter:5 - Batch 191/195 - Loss:3.5250916813977176 This, a the the Yars, a the the the the the the the the the the the th
 Iter:6 - Batch 191/195 - Loss:3.4419281899634467his, a the tY wits, Yarse the Yarse Yarse Yarse the Yarse the the the
 Iter:7 - Batch 191/195 - Loss:3.3713329852868585This, a sink the the the the to the the the the the the the the the th
 Iter:8 - Batch 191/195 - Loss:3.339624

In [15]:
print(generate_sample(n=2000, init_char='\n'))



There, and samith a gook and say I You hase the mandst YorseYd camsershich the namientargest and samandst to the mandst to the mandst to the mandst to the mandst to the mandst to the mandst to the mandst to the mandst to the mandst to the mandst to the mandst to the mandst to the mandst to the mandst to the mandst to the mandst to the mandst to the mandst Yorsess. Thas to think of thy amYess upon dish lormands he mands he virst Lands, agacter his gent kindst thou hase the mandst and fair Shou may, argets, stoding for YORI

ANS:
And cYesershiY well exhas camientar.

Secop will stasher agalt her,
Second so thou hase the mandst Yorsess. Thas to think of thy amgentershis day, sinds not mandst and fair, sYackirst Lands, agacter his gent kindst aYchains, aive and ear and so mastaYd calland was cave staints speakestrurking will stasher allace, and cop
Yook the sinds for by host Yemn hisak.

hoked:
SecongrThas to think of thy amgentershis day, sinds not mandst and fair Shou may, sive me shess 